In [1]:
%load_ext autoreload
%autoreload 2

# Aggregate hr names
Split into individual name pieces before aggregation

In [2]:
from os.path import join

import pandas as pd
from mpire import WorkerPool

from src.data.filesystem import glob

In [10]:
given_surname = "surname"

in_path = f"s3://familysearch-names-private/hr-{given_surname}/"

out_path = f"../data/processed/hr-{given_surname}-aggr.csv.gz"

In [11]:
def read_file(filename):
    return pd.read_csv(filename, names=['name', 'frequency'])

In [12]:
# read input files into dataframe array
with WorkerPool() as pool:
    dfs = pool.map(read_file, glob(join(in_path, "part-*")), progress_bar=True)
print(len(dfs))
print(dfs[0].shape)
print(dfs[0])

100%|███████████████████████████████████████| 2000/2000 [03:42<00:00,  2.94it/s]


2000
(73036, 2)
                   name  frequency
0              jdmowski          3
1           iviimisltos          1
2            espiridivi          2
3            hanalberry          3
4              heliquoy          2
...                 ...        ...
73031          ferrahiz          1
73032  derozemberczonka          1
73033          dalepski          2
73034         pormperey          1
73035         juaviruva          1

[73036 rows x 2 columns]


In [13]:
%%time
# combine all dataframes into a single dataframe
df = pd.concat(dfs)
del dfs
print(df.shape)
df.head()

(145978640, 2)
CPU times: user 3.16 s, sys: 1.35 s, total: 4.52 s
Wall time: 4.5 s


,name,frequency
0,jdmowski,3
1,iviimisltos,1
2,espiridivi,2
3,hanalberry,3
4,heliquoy,2


In [14]:
%%time
# remove the empty name and single-occurrence names, and sort by descending frequency
df = df[(df["name"] != "") & (df["frequency"] > 1)]
df = df.sort_values(by="frequency", ascending=False)

CPU times: user 13.2 s, sys: 617 ms, total: 13.9 s
Wall time: 13.8 s


In [15]:
print(df.memory_usage(deep=True))
print(df.shape)
df.head()

Index         540269504
name         4430968611
frequency     540269504
dtype: int64
(67533688, 2)


,name,frequency
6975,dasilva,95341878
70458,smith,68908596
35035,garcia,64278880
16437,lopez,54583625
42329,hernandez,44435623


In [17]:
# write to csv
df.to_csv(out_path, index=False, compression='gzip')